In [1]:
spark

Waiting for a Spark session to start...

Spark application_1540458187951_76689: Some(http://gw02.itversity.com:4047)

In [2]:
sc.getConf.getAll.filter(_._2.contains("/proxy/"))(0)._2

Waiting for a Spark session to start...

http://rm01.itversity.com:19288/proxy/application_1540458187951_76689

In [3]:
def getType(o: Any) = o.getClass.getCanonicalName

getType: (o: Any)String


In [4]:
val os_name = System.getProperty("os.name")
val hdfs_home = "/user/" + System.getenv("HOME").split("/")(2)

os_name = Linux
hdfs_home = /user/kranthidr


/user/kranthidr

In [5]:
val path = hdfs_home+"/dataSets/spark-guide/data/sample_movielens_ratings.txt"

path = /user/kranthidr/dataSets/spark-guide/data/sample_movielens_ratings.txt


/user/kranthidr/dataSets/spark-guide/data/sample_movielens_ratings.txt

In [6]:
// in Scala
import org.apache.spark.ml.recommendation.ALS
val ratings = spark.read.textFile(path)
  .selectExpr("split(value , '::') as col")
  .selectExpr(
    "cast(col[0] as int) as userId",
    "cast(col[1] as int) as movieId",
    "cast(col[2] as float) as rating",
    "cast(col[3] as long) as timestamp")

ratings = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [7]:
val Array(training, test) = ratings.randomSplit(Array(0.8, 0.2))

training = [userId: int, movieId: int ... 2 more fields]
test = [userId: int, movieId: int ... 2 more fields]


[userId: int, movieId: int ... 2 more fields]

In [8]:
val als = new ALS()
.setColdStartStrategy("drop")
  .setMaxIter(10)
  .setRegParam(0.01)
  .setUserCol("userId")
  .setItemCol("movieId")
  .setRatingCol("rating")

als = als_83775300828d


als_83775300828d

In [9]:
println(als.explainParams())

alpha: alpha for implicit preference (default: 1.0)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext (default: 10)
coldStartStrategy: strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: nan,drop. (default: nan, current: drop)
finalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK)
implicitPrefs: whether to use implicit preference (default: false)
intermediateStorageLevel: StorageLevel for intermediate datasets. Cannot be 'NONE'. (default: MEMORY_AND_DISK)
itemCol: column name for item ids. Ids must be within the integer value range. (default: item, current: movieId)
maxIter: maximum number of ite

In [10]:
val alsModel = als.fit(training)
val predictions = alsModel.transform(test)

alsModel = als_83775300828d
predictions = [userId: int, movieId: int ... 3 more fields]


[userId: int, movieId: int ... 3 more fields]

In [11]:
// COMMAND ----------

// in Scala
alsModel.recommendForAllUsers(10)
  .selectExpr("userId", "explode(recommendations)").show()

+------+---------------+
|userId|            col|
+------+---------------+
|    28| [55, 5.768714]|
|    28|[65, 5.0441747]|
|    28|[81, 4.8551626]|
|    28|[92, 4.8396864]|
|    28|[23, 4.5194364]|
|    28|[91, 4.2587166]|
|    28|[49, 4.1451116]|
|    28| [48, 4.065165]|
|    28|  [2, 3.993057]|
|    28|[46, 3.9392893]|
|    26| [51, 6.747785]|
|    26| [79, 6.030805]|
|    26| [32, 5.767578]|
|    26| [75, 5.162624]|
|    26|[98, 5.1156416]|
|    26| [24, 5.016617]|
|    26| [94, 4.996301]|
|    26|[88, 4.9798784]|
|    26| [7, 4.9670987]|
|    26| [23, 4.912221]|
+------+---------------+
only showing top 20 rows



In [12]:
alsModel.recommendForAllItems(10)
  .selectExpr("movieId", "explode(recommendations)").show()

+-------+---------------+
|movieId|            col|
+-------+---------------+
|     31|  [12, 3.64923]|
|     31|[14, 3.3269687]|
|     31| [8, 3.1149065]|
|     31|[16, 2.9235096]|
|     31|[15, 2.7864091]|
|     31| [3, 2.6915882]|
|     31| [7, 2.5365317]|
|     31|[20, 2.4639008]|
|     31| [6, 2.4038363]|
|     31|[28, 2.3084128]|
|     85| [14, 5.435953]|
|     85|[20, 5.1869564]|
|     85| [16, 4.860424]|
|     85| [8, 4.8415256]|
|     85|  [7, 4.209614]|
|     85|[19, 3.7731586]|
|     85|  [3, 3.519072]|
|     85|[24, 3.4280999]|
|     85|[17, 3.2857757]|
|     85|  [21, 2.92376]|
+-------+---------------+
only showing top 20 rows



In [13]:
// COMMAND ----------

// in Scala
import org.apache.spark.ml.evaluation.RegressionEvaluator
val evaluator = new RegressionEvaluator()
  .setMetricName("rmse")
  .setLabelCol("rating")
  .setPredictionCol("prediction")
val rmse = evaluator.evaluate(predictions)
println(s"Root-mean-square error = $rmse")

Root-mean-square error = 1.5976453222130262


evaluator = regEval_9a9c28f70bd4
rmse = 1.5976453222130262


1.5976453222130262

In [14]:
// COMMAND ----------

// in Scala
import org.apache.spark.mllib.evaluation.{
  RankingMetrics,
  RegressionMetrics}
val regComparison = predictions.select("rating", "prediction")
  .rdd.map(x => (x.getFloat(0).toDouble,x.getFloat(1).toDouble))

regComparison = MapPartitionsRDD[309] at map at <console>:42


MapPartitionsRDD[309] at map at <console>:42

In [15]:
val metrics = new RegressionMetrics(regComparison)

metrics = org.apache.spark.mllib.evaluation.RegressionMetrics@2c58ce30


org.apache.spark.mllib.evaluation.RegressionMetrics@2c58ce30

In [16]:
metrics.rootMeanSquaredError

1.5976453222130262

In [17]:
println(metrics.explainedVariance)
println(metrics.meanAbsoluteError)
println(metrics.meanSquaredError)
println(metrics.r2)
println(metrics.rootMeanSquaredError)

1.467087051966139
1.166389310090837
2.552470575589165
-0.28736460980403344
1.5976453222130262


In [18]:
// COMMAND ----------

// in Scala
import org.apache.spark.mllib.evaluation.{RankingMetrics, RegressionMetrics}
import org.apache.spark.sql.functions.{col, expr}
val perUserActual = predictions
  .where("rating > 2.5")
  .groupBy("userId")
  .agg(expr("collect_set(movieId) as movies"))

perUserActual = [userId: int, movies: array<int>]


[userId: int, movies: array<int>]

In [19]:
// COMMAND ----------

// in Scala
val perUserPredictions = predictions
  .orderBy(col("userId"), col("prediction").desc)
  .groupBy("userId")
  .agg(expr("collect_list(movieId) as movies"))

perUserPredictions = [userId: int, movies: array<int>]


[userId: int, movies: array<int>]

In [20]:
// COMMAND ----------

// in Scala
val perUserActualvPred = perUserActual.join(perUserPredictions, Seq("userId"))
  .map(row => (
    row(1).asInstanceOf[Seq[Integer]].toArray,
    row(2).asInstanceOf[Seq[Integer]].toArray.take(15)
  ))
val ranks = new RankingMetrics(perUserActualvPred.rdd)

perUserActualvPred = [_1: array<int>, _2: array<int>]
ranks = org.apache.spark.mllib.evaluation.RankingMetrics@488d82aa


org.apache.spark.mllib.evaluation.RankingMetrics@488d82aa

In [21]:
// COMMAND ----------

// in Scala
ranks.meanAveragePrecision

0.23504967254967252

In [22]:
ranks.precisionAt(5)


// COMMAND ----------

0.4769230769230769